In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('drive', force_remount=True)
from pathlib import Path
from tqdm import tqdm

!pip install surprise

from surprise import SVD #so surprise is a python scikit for building and analyzing rocommender systems - from first research this is exactly what Im going for
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate


Mounted at drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 5.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633975 sha256=55ba9f9ac6b5ec467175ba175eedb555526ffcb0ba54cfc2cf65b1ec3d4105c6
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
user_df = pd.read_csv('drive/My Drive/Colab Notebooks/data/user_ratings.csv')

In [5]:
algo = SVD()
reader = Reader(rating_scale=(1, 10))
ratings_df = pd.DataFrame()


In [6]:
user_df.head()

,Title,User_Rating,lb_username
0,Fall,2.0,colonelmortimer
1,Glorious,2.5,colonelmortimer
2,Orphan: First Kill,3.0,colonelmortimer
3,The Princess,3.0,colonelmortimer
4,Day Shift,3.5,colonelmortimer


In [5]:
# user_df[user_df['User_Rating'].isna()]

,Title,User_Rating,lb_username


In [7]:
# have to give each movie and username a unique id 
user_df['user_id']= pd.factorize(user_df.lb_username)[0]+1 #I really have to save this user 
user_df['movie_id']= pd.factorize(user_df.Title)[0]+1
#user_df['rating']=user_df['User_Rating']
user_df.shape
#user_df.to_csv("drive/My Drive/Colab Notebooks/data/user_ratings_with_ids.csv")
#ratings_df = user_df.copy(deep=True)
#user_data = user_df.drop(['Title','lb_username'], axis=1)


(99346, 5)

In [10]:
#ratings_df.astype({'rating': 'int64'})

In [8]:
print(user_df['lb_username'].nunique())

37


In [9]:
user_df[user_df['User_Rating'].isna()] #checking if the nan problem in ratings got dealt with

,Title,User_Rating,lb_username,user_id,movie_id


In [10]:
user_df['User_Rating'].isna().sum() #checking if the nan problem in ratings got dealt with

0

In [11]:
#user_df.head(8400)

In [12]:
user_df.shape

(99346, 5)

In [13]:
user_df.head()

,Title,User_Rating,lb_username,user_id,movie_id
0,Fall,2.0,colonelmortimer,1,1
1,Glorious,2.5,colonelmortimer,1,2
2,Orphan: First Kill,3.0,colonelmortimer,1,3
3,The Princess,3.0,colonelmortimer,1,4
4,Day Shift,3.5,colonelmortimer,1,5


In [ ]:
# is_user_31 = user_df['user_id']==31
# user31_df = user_df[is_user_31]
# user31_df.sort_values(by=['movie_id'],inplace=True,ascending = True)

In [ ]:
# print(user31_df)

In [14]:
print(user_df['user_id'].nunique())

37


In [15]:
data=Dataset.load_from_df(user_df[['user_id','movie_id','User_Rating']], reader)

In [22]:
#cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8720  0.8673  0.8738  0.8710  0.0027  
MAE (testset)     0.6779  0.6741  0.6745  0.6755  0.0017  
Fit time          7.69    8.40    4.54    6.88    1.68    
Test time         0.55    0.35    0.34    0.41    0.10    


{'test_rmse': array([0.87196593, 0.86728622, 0.87376844]),
 'test_mae': array([0.67788049, 0.67407114, 0.67451329]),
 'fit_time': (7.689509868621826, 8.400117874145508, 4.539899587631226),
 'test_time': (0.5547301769256592, 0.3501715660095215, 0.33952951431274414)}

In [16]:
trainSet = data.build_full_trainset()

In [17]:
algo.fit(trainSet)

In [49]:
def make_pred(uid):
  """
  make predictions for one user.
  needs int of user
  returns predictions
  """
  print('making predictions')
  anti_testset_user = []              #create empty DF
  targetUser = uid                     #30 is me right now
  fillValue=trainSet.global_mean      #have to fill missing values with something. I decided on global mean for now
  user_item_ratings=trainSet.ur[targetUser]
  user_items = [item for (item,_) in (user_item_ratings)] #here we have all the ratings from that user
  ratings = trainSet.all_ratings()
  for iid in trainSet.all_items():    #surprise generator function to iterate over all items
    if(iid not in user_items):
      anti_testset_user.append((trainSet.to_raw_uid(targetUser),trainSet.to_raw_iid(iid),fillValue))
  predictions = algo.test(anti_testset_user)
  # print(fillValue) #just out of curiosity
  return predictions

In [76]:
def sort_into_master(pred, pred_master,user_id):
  user_id = user_id+1
  df=pred_master.copy(deep=True)
  user= user_df.loc[user_df['user_id'] == user_id,'lb_username'].iloc[0]
  print(user)
  #go over each movie(iid) of the predicted DF. put the est for that user on the fitting spot(next to the movie)
  for iid in pred['iid']:
    rating_temp = pred.loc[pred.loc[pred['iid']==iid].index.values[0], 'est'] #so the position has to be found we are looking for the 'est' of the current iid in the prediction DF now we have the est as a temp which we will save into the master DF
    #index_movie = df.loc[df['movie_id']==iid].index.values #get the position of that movie in the master DF (index)

    df.loc[df['movie_id']==iid, user] =rating_temp                  #seems like this is the more clean solution. dont need an temp  index for it
    #pred_master.at[index_movie[0], user] = rating_temp                      #save the rating to the according movie

  return df

In [83]:
#pred_master = user_df[['Title', 'movie_id']]
#for user in tqdm(user_df['user_id']):
for user_id in range(3):
  user_name = user_df.loc[user_df['user_id'] == user_id+1,'lb_username'].iloc[0]
  
  if user_name in pred_master.columns:
    print('user ' + user_name + ' already in DF')
  else:
    print(user_id)
    predictions = make_pred(user_id)  ## custom function which creates anti_testset of 1 user, makes predictions for that user
    pred = pd.DataFrame(predictions) #put predictions in a DF which with we can work
    pred.sort_values(by=['iid'],inplace=True,ascending = True)
    print('user id ' + str(user_id))
    pred_temp = sort_into_master(pred, pred_master, user_id)
    pred_master = pred_temp

user colonelmortimer already in DF
user sophiedarcy already in DF
user fuchsiadyke already in DF


In [78]:
  pred_master.head(100)

,Title,movie_id,colonelmortimer,sophiedarcy,fuchsiadyke
0,Fall,1,NaN,2.727663,2.907400
1,Glorious,2,NaN,2.986660,2.530313
2,Orphan: First Kill,3,NaN,2.939237,2.917678
3,The Princess,4,NaN,2.878389,2.688069
4,Day Shift,5,NaN,2.631227,2.972056
...,...,...,...,...,...
95,Halloween Kills,95,NaN,1.982533,2.325799
96,The French Dispatch,96,NaN,NaN,3.164835
97,Dune,97,NaN,NaN,NaN
98,The Last Duel,98,NaN,NaN,2.980550


In [69]:
  pred_master.head(100)

,Title,movie_id,colonelmortimer,sophiedarcy,fuchsiadyke
0,Fall,1,NaN,2.727663,2.907400
1,Glorious,2,NaN,2.986660,2.530313
2,Orphan: First Kill,3,NaN,2.939237,2.917678
3,The Princess,4,NaN,2.878389,2.688069
4,Day Shift,5,NaN,2.631227,2.972056
...,...,...,...,...,...
95,Halloween Kills,95,NaN,1.982533,2.325799
96,The French Dispatch,96,NaN,NaN,3.164835
97,Dune,97,NaN,NaN,NaN
98,The Last Duel,98,NaN,NaN,2.980550


In [42]:
  user_id=5
  pred_master = sort_into_master(pred, pred_master,user_id)

In [26]:
#predictions = algo.test(anti_testset_user) #create predictions for not rated movies

In [91]:
# predictions[0:8]

In [28]:
# pred = pd.DataFrame(predictions) #put predictions in a DF which with we can work
# pred.sort_values(by=['iid'],inplace=True,ascending = True)

In [23]:
# pred.head()

In [22]:
# fillValue

In [24]:
# pred['est'].mean()

In [25]:
#pred.head(5000)['est']

# Goal: Sort ratings to user and movie


In [34]:
# get user from user ID
# user_id = 31
# user= user_df.loc[user_df['user_id'] == user_id,'lb_username'].iloc[0]
# user_name

'swaiborr'

In [35]:
#get user ID from username
# user_namovic = 'swaiborr'
# user_idovic = user_df.loc[user_df['lb_username'] == user_namovic,'user_id'].iloc[0]
# user_idovic

31

In [36]:
#get movie from iid
# iid=3012
# movie_name = user_df.loc[user_df['movie_id']==iid, 'Title'].iloc[0]
# movie_name

'Mad Max: Fury Road'

In [37]:
# user_df['Title'].nunique()

28783

In [39]:
# pred_master = pred_master.drop_duplicates(subset=['Title']) # first we drop duplicates. because this df is based on the user df it has duplicate movies

In [90]:
pred_master=pred_master.reset_index(drop=True) #reset the index just because I dont like the way it looks without doing this

In [77]:
# iid=1
# pred_master.loc[pred_master['movie_id']==iid].index.values
# rating_temp = pred.loc[pred.loc[pred['iid']==iid].index.values[0], 'est']
# rating_temp

3.2765537924371317

In [ ]:
# pred_master.head()

In [73]:
#pred_master[user] =np.nan

In [80]:
# user='swaiborr'
# index_movie = pred_master.loc[pred_master['movie_id']==iid].index.values
# index_movie
# pred_master.at[index_movie[0], user] = rating_temp

In [ ]:
# pred_master.head()

In [82]:
# user='swaiborr' #for each user do the anti_test_user predict --> create function for that



In [27]:
pred_master.head(100) 

,Title,movie_id,0,1,2,3,4
0,Fall,1,NaN,2.867281,2.663280,3.379652,2.763174
1,Glorious,2,NaN,3.059103,2.986796,3.304474,3.453283
2,Orphan: First Kill,3,NaN,2.968291,2.956617,3.190857,NaN
3,The Princess,4,NaN,2.806061,2.614776,2.914107,NaN
4,Day Shift,5,NaN,2.703565,2.621229,2.763378,2.836232
...,...,...,...,...,...,...,...
95,Halloween Kills,95,NaN,2.200518,1.773705,NaN,NaN
96,The French Dispatch,96,NaN,NaN,3.238053,NaN,NaN
97,Dune,97,NaN,NaN,NaN,NaN,NaN
98,The Last Duel,98,NaN,NaN,3.252471,3.709567,3.134128


so i checked my recommendations (for my user on letterboxd). I noticed that I didnt rate Dune which is a great example. I really liked the movie back then and would say that I would rate it a 4.5 or 5. the algo expects a rating of 3.92 as of today (4.9.2022) If we look at the dune movie it has a rating of 4.0. my average rating right now is 3.84. so this puts the rating right in the middle between my average and the average of the movie overall which seems fitting